In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initialize Spotify client
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='your ID ',
                                                           client_secret='your Secret'))

def get_album_release_dates(album_ids):
    album_details = {}
    for album_id in album_ids:
        album = sp.album(album_id)
        album_details[album_id] = album['release_date']
    return album_details

def get_audio_features_in_batches(track_ids, batch_size=100):
    audio_features = []
    for i in range(0, len(track_ids), batch_size):
        batch_ids = track_ids[i:i + batch_size]
        features = sp.audio_features(batch_ids)
        audio_features.extend(features)
    return audio_features

def get_artist_id(artist_name):
    results = sp.search(q=f'artist:{artist_name}', type='artist', limit=1)
    artist = results['artists']['items'][0]
    return artist['id']

def fetch_all_tracks(artist_name):
    artist_id = get_artist_id(artist_name)
    all_tracks = []
    results = sp.artist_albums(artist_id, album_type='album', limit=50)
    albums = results['items']
    
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    
    for album in albums:
        album_id = album['id']
        tracks = sp.album_tracks(album_id, limit=50)
        for track in tracks['items']:
            # Fetch track details including popularity
            track_details = sp.track(track['id'])
            all_tracks.append({
                'track_id': track['id'],
                'track_name': track['name'],
                'album_id': album_id,
                'album_name': album['name'],
                'popularity': track_details['popularity'], 
                'track_duration_ms': track_details['duration_ms']  
            })
    
    return all_tracks

def create_tracks_dataframe(tracks):
    # Extract track IDs and corresponding album IDs
    track_ids = [track['track_id'] for track in tracks]
    album_ids = list(set(track['album_id'] for track in tracks))  

    # Fetch album release dates
    album_release_dates = get_album_release_dates(album_ids)
    
    # Fetch audio features
    audio_features = get_audio_features_in_batches(track_ids)
    
    # Create a dictionary to easily map track IDs to audio features
    track_features_map = {feature['id']: feature for feature in audio_features if feature}

    # Prepare data for DataFrame
    data = []
    for track in tracks:
        track_id = track['track_id']
        album_id = track['album_id']
        features = track_features_map.get(track_id, {})
        
        # Extract all possible features
        tempo = features.get('tempo', 'No data')
        duration_ms = features.get('duration_ms', track.get('track_duration_ms', 'No data'))
        duration_s = duration_ms / 1000 if isinstance(duration_ms, (int, float)) else 'N/A'
        danceability = features.get('danceability', 'No data')
        energy = features.get('energy', 'No data')
        key = features.get('key', 'No data')
        loudness = features.get('loudness', 'No data')
        mode = features.get('mode', 'No data')
        speechiness = features.get('speechiness', 'No data')
        acousticness = features.get('acousticness', 'No data')
        instrumentalness = features.get('instrumentalness', 'No data')
        liveness = features.get('liveness', 'No data')
        valence = features.get('valence', 'No data')
        popularity = track.get('popularity', 'No data')  # Add track popularity
        
        # Deal with missing data
        if isinstance(duration_s, str):
            duration_s = 'N/A'
        
        data.append({
            'Track Name': track.get('track_name', 'No data'),
            'Album Name': track.get('album_name', 'No data'),
            'Track ID': track_id,
            'Popularity': popularity,
            'Tempo (BPM)': tempo,
            'Duration (s)': duration_s,
            'Danceability': danceability,
            'Energy': energy,
            'Key': key,
            'Loudness': loudness,
            'Mode': mode,
            'Speechiness': speechiness,
            'Acousticness': acousticness,
            'Instrumentalness': instrumentalness,
            'Liveness': liveness,
            'Valence': valence,
            'Release Date': album_release_dates.get(album_id, 'No release date available')
        })
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Fetch all tracks and create DataFrame
artist_id = 'Aphex Twin'
all_tracks = fetch_all_tracks(artist_id)
df = create_tracks_dataframe(all_tracks)

df.head()  